In [1]:
import os
os.environ["KERAS_BACKEND"] = "torch"

In [2]:
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
import numpy as np
import torch

In [3]:
# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [4]:
# Load the IMDB dataset
max_features = 10000  # Number of words to consider as features
max_len = 500  # Cut texts after this number of words
batch_size = 64
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

In [5]:
# Preprocess the data
x_train = sequence.pad_sequences(x_train, maxlen=max_len)
x_test = sequence.pad_sequences(x_test, maxlen=max_len)

In [6]:
# Move data to PyTorch tensors and to GPU if available
x_train_tensor = torch.tensor(x_train, dtype=torch.long).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float).to(device)
x_test_tensor = torch.tensor(x_test, dtype=torch.long).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float).to(device)

In [7]:
# Build the RNN model
model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))

In [8]:
# Move the model to GPU if available
model.to(device)

<Sequential name=sequential, built=False>

In [9]:
# Compile and train the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, y_train, epochs=10, batch_size=batch_size, validation_split=0.2)

Epoch 1/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 183s 585ms/step - accuracy: 0.6711 - loss: 0.5668 - val_accuracy: 0.8578 - val_loss: 0.3408
Epoch 2/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 180s 574ms/step - accuracy: 0.9123 - loss: 0.2405 - val_accuracy: 0.8839 - val_loss: 0.2800
Epoch 3/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 181s 578ms/step - accuracy: 0.9448 - loss: 0.1596 - val_accuracy: 0.8825 - val_loss: 0.3556
Epoch 4/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 179s 571ms/step - accuracy: 0.9553 - loss: 0.1294 - val_accuracy: 0.8805 - val_loss: 0.3775
Epoch 5/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 178s 570ms/step - accuracy: 0.9706 - loss: 0.0928 - val_accuracy: 0.8699 - val_loss: 0.4241
Epoch 6/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 180s 577ms/step - accuracy: 0.9715 - loss: 0.0857 - val_accuracy: 0.8475 - val_loss: 0.6078
Epoch 7/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 184s 589ms/step - accuracy: 0.9713 - loss: 0.0845 - val_accuracy: 0.8701 - val_loss: 0.4484
Epoch 8/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 184s 589ms/step - accuracy: 0.9768 -

In [10]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(x_test, y_test)
print(f'Test loss: {loss:.4f}')
print(f'Test accuracy: {accuracy:.4f}')

782/782 ━━━━━━━━━━━━━━━━━━━━ 212s 272ms/step - accuracy: 0.8559 - loss: 0.5100
Test loss: 0.5114
Test accuracy: 0.8549


In [15]:
# Select a random review from the test set
word_index = imdb.get_word_index()
reverse_word_index = {index: word for word, index in word_index.items()}

random_index = np.random.randint(0, len(x_test))
random_review = ' '.join([reverse_word_index.get(i, '?') for i in x_test[random_index] if i in reverse_word_index])


In [16]:
def predict_sentiment(review_text, model, maxlen):
    # Tokenize and pad the input sequence
    review_sequence = sequence.pad_sequences([imdb.get_word_index()[word] if word in imdb.get_word_index() else 0 for word in review_text.split()], maxlen=maxlen)

    # Make prediction
    prediction = model.predict(review_sequence)[0, 0]

    sentiment = "positive" if prediction > 0.5 else "negative"
    return sentiment

In [ ]:
print(f"\nRandom Review: {random_review}")
predicted_sentiment = predict_sentiment(random_review, model, max_len)
print(f"Predicted sentiment: {predicted_sentiment}")